In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [5]:
mnist = input_data.read_data_sets('data', one_hot=True)

tf.reset_default_graph()

batch_size = 128
num_step = 5001

def weigth_variable(shape, name):
    return tf.Variable(tf.truncated_normal(dtype=tf.float32, shape= shape, stddev=0.1), name=name)

def biase_varibale(shape, name):
    #return tf.Variable(tf.zeros(shape=shape) + 0.1)
    return tf.Variable(tf.constant(0.1, shape=shape), name=name)

def conv2d(x, W):
    return tf.nn.conv2d(input=x, filter=W, strides=[1,1,1,1], padding="SAME")

def max_pool_2x2(x):
    return tf.nn.max_pool(value=x, ksize=[1,2,2,1], strides= [1,2,2,1], padding="SAME")

def variables_summaries(var):
    mean = tf.reduce_mean(var)
    tf.summary.scalar('mean', mean)
    with tf.name_scope('stddev'):
        stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
    tf.summary.scalar('stddev', stddev)
    tf.summary.scalar('max', tf.reduce_max(var))
    tf.summary.scalar('min', tf.reduce_min(var))
    tf.summary.histogram('histogram', var)
    
#输入
with tf.name_scope(name = 'input'):
    with tf.name_scope(name = 'input_x'):
        x = tf.placeholder(dtype=tf.float32, shape=[None, 784])
    with tf.name_scope(name = 'input_y'):
        y = tf.placeholder(dtype=tf.float32, shape=[None, 10])
    with tf.name_scope(name = 'keep_prob'):
        keep_prob = tf.placeholder(dtype=tf.float32)
    with tf.name_scope(name = 'image_x'):
        image_x = tf.reshape(x, shape=[-1,28,28,1], name = 'image_x')
         
with tf.name_scope('Conv1'):
    #初始化第一个卷积层的权值和偏置值
    with tf.name_scope('W_Conv1'):
        W_Conv1 = weigth_variable([5,5,1,32], name='W_Conv1')
        variables_summaries(W_Conv1)
    with tf.name_scope('b_Conv1'):       
        b_Conv1 = biase_varibale([32], name='b_Conv1')
        variables_summaries(b_Conv1)
    #把image和权值向量卷积，再加上偏置值，然后应用relu激活函数，最后加上池化层
    with tf.name_scope('conv2d_1'):
        conv2d_1 = conv2d(image_x, W_Conv1) + b_Conv1
    with tf.name_scope('relu_1'):
        h_conv1 = tf.nn.relu(conv2d_1, name='relu_1')
    with tf.name_scope('pool_1'):
        h_pool1 = max_pool_2x2(h_conv1)
        
with tf.name_scope('Conv2'):
    #初始化第二个卷积层的权值和偏置值
    with tf.name_scope('W_Conv2'):
        W_Conv2 = weigth_variable([5,5,32,64], name='W_Conv2')
        variables_summaries(W_Conv2)
    with tf.name_scope('b_Conv2'):
        b_Conv2 = biase_varibale([64], name = 'b_Conv2')
        variables_summaries(b_Conv2)
        
    #把h_pool1和权值向量卷积，再加上偏置值，然后应用relu激活函数，最后加上池化层
    with tf.name_scope('conv2d_2'):
        conv2d_2 = conv2d(h_pool1, W_Conv2) + b_Conv2
    with tf.name_scope('relu2'):
        h_conv2 = tf.nn.relu(conv2d_2, name = 'relu2')
    with tf.name_scope('pool2'):
        h_pool2 = max_pool_2x2(h_conv2)

with tf.name_scope('fc1'):
    #初始化第一个全连接层权值
    with tf.name_scope('W_fc1'):
        W_fc1 = weigth_variable(shape=[7*7*64, 1024], name='W_fc1')
        variables_summaries(W_fc1)
    with tf.name_scope('b_fc1'):
        b_fc1 = biase_varibale(shape=[1024], name='b_fc1')
        variables_summaries(b_fc1)
        
    #h_pool2扁平化
    with tf.name_scope('h_pool2_flat'):
        h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64], name='h_pool2_flat')
    with tf.name_scope('wx_plus_b'):
        fc1 = tf.matmul(h_pool2_flat, W_fc1) + b_fc1
    with tf.name_scope('relu'):
        fc1_relu = tf.nn.relu(fc1, name='relu')
    with tf.name_scope('fc1_dropout'):
        fc1_drop = tf.nn.dropout(fc1_relu, keep_prob=keep_prob)
    
with tf.name_scope('fc2'):
    #初始化第二个全连接层权值
    with tf.name_scope('W_fc2'):
        W_fc2 = weigth_variable(shape=[1024, 10], name='W_fc2')
        variables_summaries(W_fc2)
    with tf.name_scope('b_fc2'):
        b_fc2 = weigth_variable(shape=[10], name='b_fc2')
        variables_summaries(b_fc2)
        
    with tf.name_scope('wx_plus_b'):
        fc2 = tf.matmul(fc1_drop, W_fc2) + b_fc2
    with tf.name_scope('relu'):
        fc2_relu = tf.nn.relu(fc2)
    with tf.name_scope('fc2_dropout'):
        fc2_drop = tf.nn.dropout(fc2_relu, keep_prob=keep_prob)

with tf.name_scope('fc3'):
    with tf.name_scope('W_fc3'):
        W_fc3 = weigth_variable(shape=[10,10], name='W_fc3')
        variables_summaries(W_fc3)
    with tf.name_scope('b_fc3'):
        b_fc3 = biase_varibale(shape=[10], name='b_fc3')
        variables_summaries(b_fc3)
    with tf.name_scope('wx_plus_b'):
        fc3 = tf.matmul(fc2_drop, W_fc3) + b_fc3
    
    with tf.name_scope('softmax'):
        prediction = tf.nn.softmax(fc3, name = 'softmax')

with tf.name_scope('loss'):
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=prediction,name='loss'))
    tf.summary.scalar('loss', loss)

with tf.name_scope('train'):
    train_step = tf.train.AdamOptimizer(1e-4).minimize(loss)
    
with tf.name_scope('accuracy'):
    with tf.name_scope('correction_prediction'):
        correction_prediction = tf.equal(tf.argmax(y,1), tf.argmax(prediction,1),name='correction_prediction')
    
    with tf.name_scope('accuracy'):
        accuracy = tf.reduce_mean(tf.cast(correction_prediction, dtype=tf.float32), name='accuracy')
        tf.summary.scalar('accuracy', accuracy)

merged = tf.summary.merge_all()
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    saver = tf.train.Saver()
    train_writer = tf.summary.FileWriter('cnn_logs/train', graph=sess.graph)
    test_writer = tf.summary.FileWriter('cnn_logs/test', graph=sess.graph)
 
    for epoch in range(num_step):
        xs_batch, ys_batch = mnist.train.next_batch(batch_size)     
        summary, _ = sess.run([merged,train_step], feed_dict={x:xs_batch, y:ys_batch, keep_prob:0.8})
        sess.run(train_step, feed_dict={x:xs_batch, y:ys_batch, keep_prob:0.8})
        train_writer.add_summary(summary, epoch)
        
        xs_batch, ys_batch = mnist.test.next_batch(batch_size) 
        summary = sess.run(merged, feed_dict={x:xs_batch, y:ys_batch, keep_prob:1.0})
        test_writer.add_summary(summary, epoch)
        
        if epoch % 100 == 0:
            test_acc = sess.run(accuracy, feed_dict={x:mnist.test.images, y:mnist.test.labels, keep_prob:1.0})
            train_acc = sess.run(accuracy, feed_dict={x:mnist.train.images[:10000], y:mnist.train.labels[:10000], keep_prob:1.0})
            print('Iter ' + str(epoch) + ' Training Accuracy=' + str(train_acc) +', Testing Accuracy=' + str(test_acc))
            #print('Iter ' + str(epoch) +' Testing Accuracy=' + str(test_acc))
    saver.save(sess=sess, save_path='model.ckpt')      

Extracting data/train-images-idx3-ubyte.gz
Extracting data/train-labels-idx1-ubyte.gz
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz
Iter 0 Training Accuracy=0.0957, Testing Accuracy=0.1017
Iter 100 Training Accuracy=0.6361, Testing Accuracy=0.64
Iter 200 Training Accuracy=0.7419, Testing Accuracy=0.7512
Iter 300 Training Accuracy=0.8209, Testing Accuracy=0.8298
Iter 400 Training Accuracy=0.8437, Testing Accuracy=0.8515
Iter 500 Training Accuracy=0.8544, Testing Accuracy=0.8595
Iter 600 Training Accuracy=0.8616, Testing Accuracy=0.8647
Iter 700 Training Accuracy=0.9382, Testing Accuracy=0.9434
Iter 800 Training Accuracy=0.9551, Testing Accuracy=0.9544
Iter 900 Training Accuracy=0.9569, Testing Accuracy=0.9617
Iter 1000 Training Accuracy=0.9577, Testing Accuracy=0.959
Iter 1100 Training Accuracy=0.9647, Testing Accuracy=0.9689
Iter 1200 Training Accuracy=0.9668, Testing Accuracy=0.9677
Iter 1300 Training Accuracy=0.9679, Testing Accuracy=0.9705
Iter 